<a href="https://colab.research.google.com/github/Rodrigu87/Module-2-Project-CAS-UniBe/blob/main/Divorce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **MODULE 2 PROJECT**

# The divorce code
*By Kabir, Rodrigue and Sertac*



**Research question**

This project aims to advance our understanding of the factors that contribute to divorce.

Specifically, it seeks to:


*   Predict the likelihood of divorce
*  Identify the decisive factors leading to divorce






# **Research hypotheses**



*  H1: Couples who marry at a younger age are significantly more likely to
divorce than those who marry later, even after controlling for income and education.

*   H2: Higher communication quality and trust levels strongly reduce the likelihood of divorce, while frequent conflicts increase it.

*   H3: Pre-marital cohabitation increases the risk of divorce in love marriages, but decreases the risk in arranged marriages.
*   H4: Religious compatibility between spouses reduces the probability of divorce compared to interfaith or non-aligned couples.

*   H5: Infidelity and domestic violence history are the strongest predictors of divorce, while high social support and counseling attendance mitigate these risks.
*   H6: Low trust and high conflict frequency are associated with shorter marriage duration among couples who eventually divorce.

In [ ]:
# About the dataset
# synthetic dataset from kaggle.com (https://www.kaggle.com/datasets/vanpatangan/divorce-prediction/data)


In [8]:
import pandas as pd
